In [3]:
import numpy as np
import pandas as pd

In [4]:
%%bash
wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
conda config --set always_yes yes --set changeps1 no
conda install -q -y -c conda-forge python=3.7
conda install -q -y -c conda-forge rdkit==2020.09.2 

PREFIX=/usr/local
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py38h27cfd23_1003
    - ca-certificates==2020.10.14=0
    - certifi==2020.6.20=pyhd3eb1b0_3
    - cffi==1.14.3=py38h261ae71_2
    - chardet==3.0.4=py38h06a4308_1003
    - conda-package-handling==1.7.2=py38h03888b9_0
    - conda==4.9.2=py38h06a4308_0
    - cryptography==3.2.1=py38h3c74f83_1
    - idna==2.10=py_0
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20191231=h14c3975_1
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1h=h7b6447c_0
    - pip==20.2.4=py38h06a4308_0
    - pycosat==0.6.3=py38h7b6447c_1
    - pycparser==2.20=py_2
    - pyopenssl==19.1.0=pyhd3eb1b0_1
    - pysocks==1.7.1=py38h06a4308_0
    - python==3.8.5=h7579374_1
    - readline==8.0=h7b6447c_

--2021-07-10 23:34:46--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2021-07-10 23:34:46--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94235922 (90M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 11.8M 8s
    50K .......... .......... .......... .......... ..........

In [5]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

try:
  from rdkit import Chem
except ImportError:
  print('Stopping RUNTIME. Colaboratory will restart automatically. Please run again.')
  exit()

In [6]:
import rdkit
from rdkit.Chem import PandasTools

In [7]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

#Processing mRNA Data

## Formatting mRNA Data

In [ ]:
mRNA = pd.read_csv("https://data.broadinstitute.org/ccle/CCLE_RNAseq_rsem_genes_tpm_20180929.txt.gz", compression="gzip", delimiter = "\t")

In [ ]:
columns = mRNA.columns.to_series()
columns = columns.str.split("_").str[0]
mRNA.columns = columns

In [ ]:
del mRNA["transcript"]

In [ ]:
mRNA = mRNA.transpose()

In [ ]:
mRNA.head()

gene,CELL_LINE_NAME,ENSG00000000003.10,ENSG00000000419.8,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000971.11,ENSG00000001036.9,ENSG00000001084.6,ENSG00000001167.10,ENSG00000001460.13,ENSG00000001461.12,ENSG00000001497.12,ENSG00000001561.6,ENSG00000001617.7,ENSG00000001629.5,ENSG00000001630.11,ENSG00000001631.10,ENSG00000002016.12,ENSG00000002330.9,ENSG00000002549.8,ENSG00000002586.13,ENSG00000002822.11,ENSG00000002834.13,ENSG00000002919.10,ENSG00000003056.3,ENSG00000003096.9,ENSG00000003137.4,ENSG00000003147.13,ENSG00000003249.9,ENSG00000003393.10,ENSG00000003400.10,ENSG00000003402.15,ENSG00000003436.10,ENSG00000003509.11,ENSG00000003756.12,ENSG00000003989.12,ENSG00000004059.6,ENSG00000004139.9,ENSG00000004142.7,ENSG00000004399.8,...,ENSG00000273148.1,ENSG00000273149.1,ENSG00000273151.1,ENSG00000273154.1,ENSG00000273162.1,ENSG00000273173.1,ENSG00000273184.1,ENSG00000273186.1,ENSG00000273188.1,ENSG00000273221.1,ENSG00000273226.1,ENSG00000273230.1,ENSG00000273243.1,ENSG00000273247.1,ENSG00000273261.1,ENSG00000273270.1,ENSG00000273271.1,ENSG00000273281.1,ENSG00000273290.1,ENSG00000273297.1,ENSG00000273300.1,ENSG00000273306.1,ENSG00000273308.1,ENSG00000273311.1,ENSG00000273314.1,ENSG00000273338.1,ENSG00000273344.1,ENSG00000273352.1,ENSG00000273361.1,ENSG00000273369.1,ENSG00000273373.1,ENSG00000273391.1,ENSG00000273420.1,ENSG00000273439.1,ENSG00000273449.1,ENSG00000273456.1,ENSG00000273466.1,ENSG00000273478.1,ENSG00000273488.1,ENSG00000273489.1
0,22RV1,2.650765,6.216843,3.427606,4.672991,0.111031,5.803744,6.900867,5.287251,2.611172,5.351204,5.396434,3.735522,1.238787,5.528571,7.631977,5.433961,4.649041,5.842476,5.556429,4.228819,4.334139,6.391115,4.043519,6.612942,1.903038,0.201634,5.004951,4.221104,3.353323,0.201634,3.932628,5.449231,3.654206,6.515700,3.714795,8.060318,5.170726,6.444435,4.017922,...,0.925999,2.025029,1.201634,1.937344,3.135863,6.654922,0.722466,0.722466,1.400538,2.347666,2.722466,2.978196,1.389567,2.693766,1.443607,1.321928,2.087463,3.305971,3.260026,0.411426,0.000000,2.353323,2.124328,0.910733,1.163499,0.000000,3.462052,1.859970,3.810443,1.682573,2.713696,1.189034,3.480265,2.253989,2.280956,0.641546,2.160275,3.499527,2.432959,3.174726
1,2313287,3.001802,6.781229,4.150560,3.839960,0.298658,7.425510,6.554589,4.295723,4.004501,4.907371,4.752213,4.085765,1.070389,4.671859,6.922198,5.234195,4.357552,5.927896,6.013016,5.205158,3.560715,6.360891,4.559492,6.510804,1.963474,1.565597,5.936638,3.077243,3.959770,0.782409,5.256633,0.356144,4.093391,6.160275,0.042644,6.940989,3.494416,6.042863,5.159468,...,0.565597,2.931683,1.604071,2.400538,1.948601,0.400538,0.263034,0.000000,1.655352,2.405992,2.319040,1.327687,1.718088,1.735522,2.160275,1.269033,1.695994,3.249445,2.889474,2.687061,0.000000,2.039138,1.395063,1.104337,1.084064,0.000000,3.289834,1.480265,2.720278,2.333424,2.189034,1.238787,0.000000,2.516015,1.575312,1.195348,1.384050,2.533563,1.570463,3.065228
2,253JBV,4.572890,5.845741,1.839960,3.568032,0.084064,4.990047,4.533563,3.938286,2.480265,3.161888,6.628774,2.217231,1.541019,4.400538,1.799087,5.346957,4.596339,5.546277,5.198887,5.153400,3.986411,6.659211,4.102658,6.439956,2.903038,3.460743,2.459432,2.910733,3.531069,2.336283,7.128768,6.506050,3.762349,5.569552,0.014355,8.032762,0.956057,6.457791,4.116032,...,1.090853,2.715893,1.985500,2.451541,0.815575,6.403097,1.500802,1.372952,1.427606,1.883621,1.232661,1.536053,0.948601,0.887525,0.925999,0.765535,0.933573,2.853996,2.659925,1.130931,0.757023,1.560715,2.241840,0.650765,2.207893,3.458119,3.246408,1.378512,2.028569,1.150560,1.739848,1.361768,2.070389,2.100978,1.195348,1.014355,1.687061,2.150560,1.063503,2.341986
3,253J,4.577731,5.535742,2.087463,2.646163,0.356144,5.525129,4.354029,3.155425,2.226509,3.471187,6.033423,2.277985,3.632268,3.693766,2.618239,4.931683,4.150560,5.521365,4.924575,5.681168,3.674687,5.800123,3.626439,6.316689,6.151372,1.691534,1.416840,2.580145,2.611172,2.339137,5.571677,7.840904,3.432959,5.171527,0.056584,7.637784,0.739848,6.075960,3.190615,..

In [ ]:
mRNA.columns = mRNA.iloc[0]
mRNA = mRNA.drop("gene")

In [ ]:
mRNA = np.log2((mRNA+1).astype("float64"))

In [ ]:
mRNA= mRNA.loc[:, mRNA.mean() > 1]

In [ ]:
mRNA = mRNA.loc[:, mRNA.std() > .25]

In [ ]:
TCGA_mRNA = pd.read_csv("https://toil-xena-hub.s3.us-east-1.amazonaws.com/download/tcga_RSEM_gene_tpm.gz", compression="gzip", delimiter="\t")
TCGA_mRNA = TCGA_mRNA.set_index(TCGA_mRNA.iloc[:, 0].name, drop=True)
TCGA_mRNA = TCGA_mRNA.transpose().astype("float64")
TCGA_mRNA = 2**(TCGA_mRNA) - 0.001
TCGA_mRNA = np.log2(TCGA_mRNA+1)
TCGA_mRNA= TCGA_mRNA.loc[:, TCGA_mRNA.mean() > 1]
TCGA_mRNA = TCGA_mRNA.loc[:, TCGA_mRNA.std() > .25]

In [ ]:
TCGA_mRNA.head()

In [ ]:
mRNA = mRNA[list(mRNA.columns & TCGA_mRNA.columns)]
TCGA_mRNA = TCGA_mRNA[list(mRNA.columns & TCGA_mRNA.columns)]

In [ ]:
mRNA.reset_index(inplace=True)
mRNA.rename(columns={"index":"CELL_LINE_NAME"}, inplace=True)

## Preprocessing TCGA and CCLE Data for Autoencoder pretraining and testing

In [ ]:
TCGA_mRNA_GeneRobustScaled = TCGA_mRNA.copy()
TCGA_mRNA_GeneMinMaxScaled = TCGA_mRNA.copy()
TCGA_mRNA_GeneRobustScaled.iloc[:,:] = RobustScaler().fit_transform(TCGA_mRNA_GeneRobustScaled.iloc[:,:])
TCGA_mRNA_GeneMinMaxScaled.iloc[:,:] = MinMaxScaler().fit_transform(TCGA_mRNA_GeneMinMaxScaled.iloc[:,:])

In [ ]:
TCGA_mRNA_LineRobustScaled = TCGA_mRNA.transpose(copy = True)
TCGA_mRNA_LineMinMaxScaled = TCGA_mRNA_LineRobustScaled.copy()
TCGA_mRNA_LineRobustScaled.iloc[:,:] = RobustScaler().fit_transform(TCGA_mRNA_LineRobustScaled.iloc[:,:])
TCGA_mRNA_LineMinMaxScaled.iloc[:,:] = MinMaxScaler().fit_transform(TCGA_mRNA_LineMinMaxScaled.iloc[:,:])
TCGA_mRNA_LineRobustScaled = TCGA_mRNA_LineRobustScaled.transpose()
TCGA_mRNA_LineMinMaxScaled = TCGA_mRNA_LineMinMaxScaled.transpose()

In [ ]:
TCGA_mRNA_LineRobustScaled.to_csv("TCGA_mRNA_LineRobustScaled.csv")
TCGA_mRNA_GeneRobustScaled.to_csv("TCGA_mRNA_GeneRobustScaled.csv")
TCGA_mRNA_LineMinMaxScaled.to_csv("TCGA_mRNA_LineMinMaxScaled.csv")
TCGA_mRNA_GeneMinMaxScaled.to_csv("TCGA_mRNA_GeneMinMaxScaled.csv")

In [ ]:
TCGA_mRNA.to_csv("TCGA_mRNA_Unscaled.csv")

In [ ]:
mRNA.set_index("CELL_LINE_NAME")

In [ ]:
mRNA_GeneRobustScaled = mRNA.copy()
mRNA_GeneMinMaxScaled = mRNA.copy()
mRNA_GeneRobustScaled.iloc[:,:] = RobustScaler().fit_transform(mRNA_GeneRobustScaled.iloc[:,:])
mRNA_GeneMinMaxScaled.iloc[:,:] = MinMaxScaler().fit_transform(mRNA_GeneMinMaxScaled.iloc[:,:])

In [ ]:
mRNA_LineRobustScaled = mRNA.transpose(copy = True)
mRNA_LineMinMaxScaled = mRNA_LineRobustScaled.copy()
mRNA_LineRobustScaled.iloc[:,:] = RobustScaler().fit_transform(mRNA_LineRobustScaled.iloc[:,:])
mRNA_LineMinMaxScaled.iloc[:,:] = MinMaxScaler().fit_transform(mRNA_LineMinMaxScaled.iloc[:,:])
mRNA_LineRobustScaled = mRNA_LineRobustScaled.transpose()
mRNA_LineMinMaxScaled = mRNA_LineMinMaxScaled.transpose()

In [ ]:
mRNA_LineRobustScaled.to_csv("mRNA_LineRobustScaled.csv")
mRNA_GeneRobustScaled.to_csv("mRNA_GeneRobustScaled.csv")
mRNA_LineMinMaxScaled.to_csv("mRNA_LineMinMaxScaled.csv")
mRNA_GeneMinMaxScaled.to_csv("mRNA_GeneMinMaxScaled.csv")

In [ ]:
mRNA.to_csv("mRNA_Unscaled.csv")

# Processing MiRNA Data

In [ ]:
miRNA = pd.read_csv("https://data.broadinstitute.org/ccle/CCLE_miRNA_20181103.gct", delimiter="\t", skiprows=2)

In [ ]:
miRNA = miRNA.set_index('Description')
miRNA = miRNA.drop('Name', 1)
miRNA = miRNA.T

In [ ]:
miRNA = np.log2(miRNA+1)
miRNA= miRNA.loc[:, miRNA.mean() > 1]
miRNA = miRNA.loc[:, miRNA.std() > .25]

In [ ]:
miRNA_GeneRobustScaled = miRNA.copy()
miRNA_GeneMinMaxScaled = miRNA.copy()
miRNA_GeneRobustScaled.iloc[:,:] = RobustScaler().fit_transform(miRNA_GeneRobustScaled.iloc[:,:])
miRNA_GeneMinMaxScaled.iloc[:,:] = MinMaxScaler().fit_transform(miRNA_GeneMinMaxScaled.iloc[:,:])

In [ ]:
miRNA_LineRobustScaled = miRNA.transpose(copy = True)
miRNA_LineMinMaxScaled = miRNA_LineRobustScaled.copy()
miRNA_LineRobustScaled.iloc[:,:] = RobustScaler().fit_transform(miRNA_LineRobustScaled.iloc[:,:])
miRNA_LineMinMaxScaled.iloc[:,:] = MinMaxScaler().fit_transform(miRNA_LineMinMaxScaled.iloc[:,:])
miRNA_LineRobustScaled = miRNA_LineRobustScaled.transpose()
miRNA_LineMinMaxScaled = miRNA_LineMinMaxScaled.transpose()

In [ ]:
miRNA_LineRobustScaled.to_csv("miRNA_LineRobustScaled.csv")
miRNA_GeneRobustScaled.to_csv("miRNA_GeneRobustScaled.csv")
miRNA_LineMinMaxScaled.to_csv("miRNA_LineMinMaxScaled.csv")
miRNA_GeneMinMaxScaled.to_csv("miRNA_GeneMinMaxScaled.csv")

In [ ]:
miRNA.to_csv("miRNA_Unscaled.csv")

# Processing Protein Data

In [ ]:
Protein = pd.read_csv("/content/drive/MyDrive/Unprocessed Data/CCLE_RPPA_20181003 (1).csv", index_col=0)

In [ ]:
Protein_GeneRobustScaled = Protein.copy()
Protein_GeneMinMaxScaled = Protein.copy()
Protein_GeneRobustScaled.iloc[:,:] = RobustScaler().fit_transform(Protein_GeneRobustScaled.iloc[:,:])
Protein_GeneMinMaxScaled.iloc[:,:] = MinMaxScaler().fit_transform(Protein_GeneMinMaxScaled.iloc[:,:])
Protein_LineRobustScaled = Protein.transpose(copy = True)
Protein_LineMinMaxScaled = Protein_LineRobustScaled.copy()
Protein_LineRobustScaled.iloc[:,:] = RobustScaler().fit_transform(Protein_LineRobustScaled.iloc[:,:])
Protein_LineMinMaxScaled.iloc[:,:] = MinMaxScaler().fit_transform(Protein_LineMinMaxScaled.iloc[:,:])
Protein_LineRobustScaled = Protein_LineRobustScaled.transpose()
Protein_LineMinMaxScaled = Protein_LineMinMaxScaled.transpose()
Protein_LineRobustScaled.to_csv("Protein_LineRobustScaled.csv")
Protein_GeneRobustScaled.to_csv("Protein_GeneRobustScaled.csv")
Protein_LineMinMaxScaled.to_csv("Protein_LineMinMaxScaled.csv")
Protein_GeneMinMaxScaled.to_csv("Protein_GeneMinMaxScaled.csv")
Protein.to_csv("Protein_Unscaled.csv")

# Preparing Regression Data


In [1]:
mRNA_Unscaled = pd.read_csv("/content/drive/MyDrive/Processed Data/Processed mRNA/mRNA_Unscaled.csv")

NameError: ignored

In [2]:
miRNA_Unscaled = pd.read_csv("/content/drive/MyDrive/Processed Data/Processed miRNA/miRNA_Unscaled.csv")
Protein_Unscaled = pd.read_csv("/content/drive/MyDrive/Processed Data/Processed Protein/Protein_Unscaled.csv")

NameError: ignored

In [ ]:
Drugs = pd.read_csv("/content/drive/MyDrive/Processed Data/Processed Drug Response Data/Processed_Drug_Response.csv")

In [ ]:
Drugs.CELL_LINE_NAME = Drugs.CELL_LINE_NAME.str.replace("-", "")

In [ ]:
CellLines = Drugs[['CELL_LINE_NAME']].copy()

In [ ]:
mRNA_Unscaled.rename(columns={mRNA_Unscaled.columns[0]: "CELL_LINE_NAME" }, inplace = True)
miRNA_Unscaled.rename(columns={miRNA_Unscaled.columns[0]: "CELL_LINE_NAME" }, inplace = True)
Protein_Unscaled.rename(columns={Protein_Unscaled.columns[0]: "CELL_LINE_NAME" }, inplace = True)

In [ ]:
mRNA_Unscaled.CELL_LINE_NAME = mRNA_Unscaled.CELL_LINE_NAME.str.split("_").str[0]
miRNA_Unscaled.CELL_LINE_NAME = miRNA_Unscaled.CELL_LINE_NAME.str.split("_").str[0]
Protein_Unscaled.CELL_LINE_NAME = Protein_Unscaled.CELL_LINE_NAME.str.split("_").str[0]

In [ ]:
CellLines = list(set(mRNA_Unscaled.CELL_LINE_NAME.unique()) & set(Protein_Unscaled.CELL_LINE_NAME.unique()) & set(miRNA_Unscaled.CELL_LINE_NAME.unique()) & set(CellLines.CELL_LINE_NAME.unique()))

In [ ]:
New_Drugs_Responses= Drugs[Drugs.CELL_LINE_NAME.isin(CellLines)].sort_values("CELL_LINE_NAME")
New_mRNA= mRNA_Unscaled[mRNA_Unscaled.CELL_LINE_NAME.isin(CellLines)].sort_values("CELL_LINE_NAME")

In [ ]:
New_miRNA= miRNA_Unscaled[miRNA_Unscaled.CELL_LINE_NAME.isin(CellLines)].sort_values("CELL_LINE_NAME")
New_Protein= Protein_Unscaled[Protein_Unscaled.CELL_LINE_NAME.isin(CellLines)].sort_values("CELL_LINE_NAME")

In [ ]:
CellLines = New_Drugs_Responses[['CELL_LINE_NAME']].copy()

In [ ]:
New_mRNA = New_mRNA.drop_duplicates(subset="CELL_LINE_NAME")

In [ ]:
New_miRNA = New_miRNA.drop_duplicates(subset="CELL_LINE_NAME")
New_Protein = New_Protein.drop_duplicates(subset="CELL_LINE_NAME")

In [ ]:
CellLines = CellLines.assign(cnt=CellLines.groupby('CELL_LINE_NAME').cumcount())

In [ ]:
New_mRNA = New_mRNA.assign(cnt=New_mRNA.groupby('CELL_LINE_NAME').cumcount())
New_mRNA = CellLines.merge(New_mRNA, on=['CELL_LINE_NAME', 'cnt'], how='left').drop('cnt', 1)
New_mRNA = New_mRNA.set_index('CELL_LINE_NAME')

In [ ]:
New_miRNA = New_miRNA.assign(cnt=New_miRNA.groupby('CELL_LINE_NAME').cumcount())
New_miRNA = CellLines.merge(New_miRNA, on=['CELL_LINE_NAME', 'cnt'], how='left').drop('cnt', 1)
New_miRNA = New_miRNA.set_index('CELL_LINE_NAME')

In [ ]:
New_Protein = New_Protein.assign(cnt=New_Protein.groupby('CELL_LINE_NAME').cumcount())
New_Protein = CellLines.merge(New_Protein, on=['CELL_LINE_NAME', 'cnt'], how='left').drop('cnt', 1)
New_Protein = New_Protein.set_index('CELL_LINE_NAME')

In [ ]:
New_mRNA.to_csv("New_mRNA_Full.csv")
New_miRNA.to_csv("New_miRNA.csv")
New_Protein.to_csv("New_Protein.csv")

## Getting Fingerprints for Drugs

In [ ]:
 PandasTools.AddMoleculeColumnToFrame(New_Drugs_Responses, smilesCol='DRUG_NAME', molCol='Fingerprints', includeFingerprints=True)

In [ ]:
radius=3
nBits=1024
ECFP6 = [rdkit.DataStructs.cDataStructs.BitVectToText(Chem.AllChem.GetMorganFingerprintAsBitVect(x,radius=radius, nBits=nBits)) for x in New_Drugs_Responses['Fingerprints']]

In [ ]:
New_Drugs_Responses["Fingerprints"] = ECFP6

In [ ]:
New_Drugs_Responses = New_Drugs_Responses.rename(columns={'DRUG_NAME':'SMILES'})

In [ ]:
New_Drugs_Responses.drop(columns=New_Drugs_Responses.columns[0], 
        axis=1, 
        inplace=True)

In [ ]:
arr = []

In [ ]:
for index, row in New_Drugs_Responses.iterrows():
    arr.append(np.fromiter(row["Fingerprints"], dtype=np.int32).tolist())

In [ ]:
arr = np.array(arr)

In [ ]:
Fingerprints = pd.DataFrame(arr)

In [ ]:
Fingerprints.to_csv("Fingerprints.csv")

In [ ]:
New_Drugs_Responses.to_csv("New_Drug_Responses.csv")